In [ ]:
import pandas as pd
import json
import matplotlib.pyplot as plt
import seaborn as sns
import os

# Pandas settings
pd.set_option('display.max_columns', None)
pd.set_option('display.width', 1000)

# CONFIGfolder
RAW_DATA_PATH = "../data/raw/dataset_COMMAND_USAGE_raw.csv"
CLEAN_DATA_PATH = "../data/dataset_commands_clean.csv"

print("[+] Configuration loaded.")

In [ ]:
# LOAD DATA
if os.path.exists(RAW_DATA_PATH):
    df = pd.read_csv(RAW_DATA_PATH)
    print(f"[+] Loaded {len(df)} command logs.")
else:
    print(f"[-] File not found: {RAW_DATA_PATH}. Please run download_data.ipynb first.")
    df = pd.DataFrame()

if not df.empty:
    # 2. JSON EXPANSION
    def clean_json(x):
        if isinstance(x, dict): return x
        try:
            return json.loads(x)
        except:
            return {}


    df['context_data'] = df['context_data'].apply(clean_json)
    df_context = pd.json_normalize(df['context_data'])

    # Remove duplicated columns (world, biome, timestamp, playerUuid)
    # playerUuid usually comes in the main table as player_uuid
    cols_to_drop = df_context.columns.intersection(df.columns)
    if not cols_to_drop.empty:
        df_context = df_context.drop(columns=cols_to_drop)

    # Join
    df_final = df.join(df_context).drop(columns=['context_data'])

    # FIX: Check if 'command' column exists (it should come from JSON)
    if 'command' not in df_final.columns:
        print("[-] Warning: 'command' column missing from JSON data.")

    print("[+] JSON expanded successfully.")
    display(df_final.head(3))

In [ ]:
# FEATURE ENGINEERING (The Logic)

if not df_final.empty and 'command' in df_final.columns:
    # A. Extract Base Command
    # We want "/tpa Player123" -> "/tpa"
    # We split by space and take the first part
    df_final['base_command'] = df_final['command'].astype(str).apply(lambda x: x.split(' ')[0])

    # B. Identify Teleportation Commands
    # Useful to know if the player is an "Explorer" (walking) or "Fast Traveler"
    teleport_keywords = ['/home', '/warp', '/tpa', '/tpaccept', '/back', '/spawn', '/rtp']
    df_final['is_teleport'] = df_final['base_command'].isin(teleport_keywords)

    # C. Server ID (One-Hot)
    if 'server_id' in df_final.columns:
        server_dummies = pd.get_dummies(df_final['server_id'], prefix='server')
        df_final = pd.concat([df_final, server_dummies], axis=1)

    # D. Command Length (Are they typing long messages/arguments?)
    df_final['cmd_length'] = df_final['command'].astype(str).apply(len)

    print("[+] Feature Engineering complete.")
    display(df_final[['command', 'base_command', 'is_teleport', 'isSuccess']].head())

In [ ]:
# DATA ANALYSIS & VISUALIZATION

if not df_final.empty:
    # Graph 1: Top 10 Most Used Commands
    plt.figure(figsize=(10, 6))
    top_commands = df_final['base_command'].value_counts().head(10).index

    sns.countplot(
        y='base_command',
        data=df_final[df_final['base_command'].isin(top_commands)],
        order=top_commands,
        palette='magma',
        hue='base_command',
        legend=False
    )
    plt.title('Top 10 Most Used Commands')
    plt.xlabel('Count')
    plt.ylabel('Command')
    plt.show()

    # Graph 2: Success Rate
    if 'isSuccess' in df_final.columns:
        plt.figure(figsize=(5, 4))
        sns.countplot(x='isSuccess', data=df_final, palette='viridis', hue='isSuccess', legend=False)
        plt.title('Command Success Rate (Typos vs Valid)')
        plt.show()

In [ ]:
# SAVE CLEAN DATASET

if not df_final.empty:
    # Clean up unnecessary columns
    cols_to_keep = [
        'base_command',
        'isSuccess',
        'is_teleport',
        'cmd_length',
        'world',
        'biome'
    ]
    # Add server columns dynamically
    cols_to_keep.extend([col for col in df_final.columns if 'server_' in col])

    df_export = df_final[cols_to_keep].copy()

    df_export.to_csv(CLEAN_DATA_PATH, index=False)
    print(f"[+] Clean commands dataset saved to: {CLEAN_DATA_PATH}")
    display(df_export.head())